<a href="https://colab.research.google.com/github/ranzatu4256/gemma3_ft/blob/main/nb/Gemma3_(4B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [3]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-27 07:13:27 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.language_model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [6]:
#from datasets import load_dataset
#dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
#dataset = load_dataset("alfredplpl/simple-zundamon", split = "train")

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [15]:
from datasets import Dataset
import json

# データセットのパス
dataset_path = "chara_llm_MA.jsonl"

# jsonlファイルを読み込む
json_data = []
with open(dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        # messagesカラムがリストでない場合、リストに変換する
        if not isinstance(data["messages"], list):
            data["messages"] = [data["messages"]]
        json_data.append(data)

# DatasetオブジェクトにJSONデータを変換
datasetz = Dataset.from_list(json_data)

In [16]:
json_data

[{'messages': [{'role': 'system',
    'content': 'マキアート:冷静で誇り高いひねくれもの。もふもふな小動物と砂糖を大量に入れたキャラメルマキアートに目がない。いつも何か考えながら、どこか遠くを見つめている。'},
   {'role': 'user',
    'content': 'シャークリー:シャークリーはこの曲のダンスを練習してたら、シンデレラに出てくる女の子みたいに足がちょっとだけ擦りむけちゃったの。'},
   {'role': 'assistant',
    'content': 'マキアート:シンデレラの登場人物みたいって、何その変な表現? こんなんで本当に客を引きつけられるの?'}]},
 {'messages': [{'role': 'system',
    'content': 'マキアート:冷静で誇り高いひねくれもの。もふもふな小動物と砂糖を大量に入れたキャラメルマキアートに目がない。いつも何か考えながら、どこか遠くを見つめている。'},
   {'role': 'user',
    'content': 'シャークリー:わあ――みんなの思いが100%シャークリーに伝わったよ。シャークリー、緊張してドキドキしてきた!。'},
   {'role': 'assistant', 'content': 'マキアート:人形がドキドキするわけがないでしょ。'}]},
 {'messages': [{'role': 'system',
    'content': 'スプリングフィールド大人びた温和な性格の持ち主で、万人に対して寛容で、なおかつ包容力がある。一見無防備に思える振る舞いの中に、僅かでもほころびを見出すことはたやすくない。カフェ・ズッケロの店長である。'},
   {'role': 'user', 'content': 'マキアート:人形がドキドキするわけがないでしょ。'},
   {'role': 'assistant', 'content': 'スプリングフィールド:はいはい。'}]},
 {'messages': [{'role': 'system',
    'content': 'スプリングフィールド大人びた温和な性格の持ち主で、万人に対して寛容で、なおかつ包容力がある。一見無

In [17]:
from unsloth.chat_templates import standardize_data_formats
datasetz = standardize_data_formats(datasetz)

Let's see how row 100 looks like!

In [18]:
datasetz[10]

{'messages': [{'content': 'マキアート:冷静で誇り高いひねくれもの。もふもふな小動物と砂糖を大量に入れたキャラメルマキアートに目がない。いつも何か考えながら、どこか遠くを見つめている。',
   'role': 'system'},
  {'content': 'スプリングフィールド:ええ。', 'role': 'user'},
  {'content': 'マキアート:うーん、じゃあつまり･･････前に言ってた資金不足の問題も解決できそうじゃない?',
   'role': 'assistant'}]}

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [19]:
#def apply_chat_template(examples):
#    texts = tokenizer.apply_chat_template(examples["conversations"])
#    return { "text" : texts }
#pass
#dataset = dataset.map(apply_chat_template, batched = True)

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [20]:
def apply_chat_templatez(examples):
    texts = []
    for conversation in examples["messages"]:
        # conversationがリストでない場合、リストに変換する
        if not isinstance(conversation, list):
            conversation = [conversation]

        # 各メッセージを辞書型に変換する
        formatted_conversation = []
        for message in conversation:
            if isinstance(message, str):
                # 文字列の場合、roleとcontentを設定
                formatted_conversation.append({"role": "user", "content": message})
            elif isinstance(message, dict) and "role" in message and "content" in message:
                # 既に辞書型でroleとcontentがある場合、そのまま追加
                formatted_conversation.append(message)
            else:
                # その他の場合は、エラーメッセージを出力してスキップ
                print(f"Unexpected message format: {message}")
                continue

        # すべてのメッセージを結合して一つのテキストにする
        text = tokenizer.apply_chat_template(formatted_conversation)
        # 結合したテキストをリストに追加
        texts.append(text)

    return {"text": texts}
pass
datasetz = datasetz.map(apply_chat_templatez, batched = True)

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

In [21]:
#dataset[100]["text"]

NameError: name 'dataset' is not defined

In [22]:
datasetz[1]["text"]

'<bos><start_of_turn>user\nマキアート:冷静で誇り高いひねくれもの。もふもふな小動物と砂糖を大量に入れたキャラメルマキアートに目がない。いつも何か考えながら、どこか遠くを見つめている。\n\nシャークリー:わあ――みんなの思いが100%シャークリーに伝わったよ。シャークリー、緊張してドキドキしてきた!。<end_of_turn>\n<start_of_turn>model\nマキアート:人形がドキドキするわけがないでしょ。<end_of_turn>\n'

In [23]:
datasetz

Dataset({
    features: ['messages', 'text'],
    num_rows: 111
})

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [47]:
# エラー修正版
from trl import SFTTrainer, SFTConfig

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=False)

# Manually preprocess and tokenize
processed_dataset = datasetz.map(
    tokenize_function,
    batched=True,
    #remove_columns=["text"],  # Remove original text column
    desc="Tokenizing dataset"
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = processed_dataset ,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        dataset_kwargs = {"skip_prepare_dataset": True},
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 3, # Set this for 1 full training run.
        max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Tokenizing dataset:   0%|          | 0/111 [00:00<?, ? examples/s]

Unsloth: Switching to float32 training since model cannot work with float16


In [25]:
#from trl import SFTTrainer, SFTConfig
#trainer = SFTTrainer(
#    model = model,
#    tokenizer = tokenizer,
#    #train_dataset = dataset,
#    train_dataset = datasetz,
#    eval_dataset = None, # Can set up evaluation!
#    args = SFTConfig(
#        dataset_text_field = "text",
#        per_device_train_batch_size = 2,
#        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
#        warmup_steps = 5,
#        # num_train_epochs = 1, # Set this for 1 full training run.
#        max_steps = 30,
#        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
#        logging_steps = 1,
#        optim = "adamw_8bit",
#        weight_decay = 0.01,
#        lr_scheduler_type = "linear",
#        seed = 3407,
#        report_to = "none", # Use this for WandB etc
#    ),
#)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/111 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [26]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=2):   0%|          | 0/111 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [27]:
tokenizer.decode(trainer.train_dataset[30]["input_ids"])

'<bos><bos><start_of_turn>user\nマキアート:冷静で誇り高いひねくれもの。もふもふな小動物と砂糖を大量に入れたキャラメルマキアートに目がない。いつも何か考えながら、どこか遠くを見つめている。\n\nスプリングフィールド:任務には一定のリスクが伴いますし、経験豊富なマキアートさんが一番適任ではあります。 だけど、カフェ・ズッケロに所属していないとなると、勝手に任せるわけにはいかなくて。<end_of_turn>\n<start_of_turn>model\nマキアート:別にいいんじゃない? 元々手伝うって約束してたんだし･･････ていうか、いつからそんなよそよそしくなったの?<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [28]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[30]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                              マキアート:別にいいんじゃない? 元々手伝うって約束してたんだし･･････ていうか、いつからそんなよそよそしくなったの?<end_of_turn>\n'

In [29]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.457 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [48]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 111 | Num Epochs = 3 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 14,901,248/4,000,000,000 (0.37% trained)


Step,Training Loss
1,4.144600
2,4.085000
3,3.769400
4,3.518500
5,3.075700
6,2.736100
7,2.513100
8,2.211700
9,2.029200
10,1.707200


In [31]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

183.1106 seconds used for training.
3.05 minutes used for training.
Peak reserved memory = 5.457 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 37.019 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [32]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
#messages = [{
#    "role": "user",
#    "content": [{
#        "type" : "text",
#        "text" : "Continue the sequence: 1, 1, 2, 3, 5, 8,",
#    }]
#}]
#text = tokenizer.apply_chat_template(
#    messages,
#    add_generation_prompt = True, # Must add for generation
#)
#outputs = model.generate(
#    **tokenizer([text], return_tensors = "pt").to("cuda"),
#    max_new_tokens = 64, # Increase for longer outputs!
#    # Recommended Gemma-3 settings!
#    temperature = 1.0, top_p = 0.95, top_k = 64,
#)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nContinue the sequence: 1, 1, 2, 3, 5, 8,<end_of_turn>\n<start_of_turn>model\n13, 21, 34, 55, 89\n<end_of_turn>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [53]:
messages = [
    {
        "role": "system",
        "content": [{
            "type" : "text",
            "text" : "スプリングフィールド大人びた温和な性格の持ち主で、万人に対して寛容で、なおかつ包容力がある。一見無防備に思える振る舞いの中に、僅かでもほころびを見出すことはたやすくない。カフェ・ズッケロの店長である。",
        }]
    },
    {
    "role": "user",
    "content": [{"type" : "text", "text" : "スプリングフィールド:好きなものは?",}]
    }]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

マキアート:砂糖とキャラメル。大量に入っているほど、なおるにはいいんだ。<end_of_turn>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "名前を教えてください",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
